In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Imports

In [2]:
import os
import sys
sys.path.append('/home/rcendre/classification')

import itertools
import pandas
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from misvm import SIL, MISVM
from numpy import array, logspace
from scipy.stats import randint,uniform
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.models.models import CustomMIL, MultimodalClassifier
from toolbox.models.builtin import Applications
from toolbox.IO import dermatology
from sklearn.metrics import f1_score
from toolbox.transforms.common import PredictorTransform, FlattenTransform, LinearTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
import warnings
warnings.filterwarnings('ignore') 

Using TensorFlow backend.


# Parameters

In [3]:
# Advanced parameters
data_type='Full'
validation = 4
settings = Settings.get_default_dermatology()

# Photography

In [4]:
p_inputs = IO.load('Photography.pickle')

In [5]:
extractor = 'InceptionResNetV2Avg'

# SVM Linear
model = Pipeline([('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
model_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [6]:
pred_folds = [([1,2], [3]),
             ([2,3], [4]),
             ([3,4], [1]),
             ([4,1], [2])]
Tools.evaluate(p_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=pred_folds, distribution=model_params) 

In [7]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(p_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy'])))

,precision,recall,f1-score,support
Benign,0.41±0.12,0.58±0.17,0.48±0.10,86.00±6.18
Malignant,0.65±0.08,0.49±0.14,0.56±0.09,138.00±4.56
accuracy,0.52±0.06,0.52±0.06,0.52±0.06,0.52±0.06
macro avg,0.53±0.04,0.53±0.04,0.52±0.06,224.00±4.30
weighted avg,0.56±0.03,0.52±0.06,0.53±0.07,224.00±4.30


# Dermoscopy

In [8]:
d_inputs = IO.load('Dermoscopy.pickle')
d_inputs = pandas.concat([p_inputs, d_inputs], axis=0)

In [9]:
extractor = 'InceptionResNetV2Avg'

# SVM Linear
model = Pipeline([('flat', FlattenTransform()), ('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
model_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [10]:
all_image = [True] * len(d_inputs.index)
single_image = d_inputs['Modality'] == 'Photography'
Data.build_bags(d_inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', extractor)
d_inputs = d_inputs[single_image].reset_index()

In [11]:
pred_folds = [([1,2], [3]),
             ([2,3], [4]),
             ([3,4], [1]),
             ([4,1], [2])]
Tools.evaluate(d_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=pred_folds, distribution=model_params) 

In [12]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(d_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy'])))

,precision,recall,f1-score,support
Benign,0.54±0.15,0.58±0.10,0.56±0.08,86.00±6.18
Malignant,0.73±0.09,0.69±0.11,0.71±0.06,138.00±4.56
accuracy,0.65±0.07,0.65±0.07,0.65±0.07,0.65±0.07
macro avg,0.63±0.06,0.63±0.07,0.63±0.07,224.00±4.30
weighted avg,0.65±0.06,0.65±0.07,0.65±0.06,224.00±4.30


# Microscopy

In [13]:
m_inputs = IO.load('Microscopy.pickle')

In [14]:
extractor = 'ResNetAvg'
low_folds = [([1], [2]),
             ([2], [3]),
             ([3], [4]),
             ([4], [1])]
Tools.fit_predict(m_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, MinMaxScaler(), 'Scaled', folds=low_folds)

In [15]:
all_image = [True] * len(m_inputs.index)
single_image = m_inputs['ID_Image'] == '0M'
Data.build_bags(m_inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', extractor)
Data.build_bags(m_inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', 'Scaled')
m_inputs = m_inputs[single_image].reset_index()

In [16]:
Tools.transform(m_inputs, {'datum': 'Scaled'}, LinearTransform(method='average'), 'ScaledAvg')
Tools.transform(m_inputs, {'datum': 'Scaled'}, LinearTransform(method='max'), 'ScaledMax')
Tools.transform(m_inputs, {'datum': 'Scaled'}, LinearTransform(method=4), 'ScaledNorm4')
Tools.transform(m_inputs, {'datum': 'Scaled'}, LinearTransform(method=6), 'ScaledNorm6')
Tools.transform(m_inputs, {'datum': 'Scaled'}, LinearTransform(method=8), 'ScaledNorm8')

In [17]:
pred_folds = [([2], [3]),
                ([3], [4]),
                ([4], [1]),
                ([1], [2])]

model = Pipeline([('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
model_params = {'clf__C': logspace(-2, 3, 6).tolist()}

Tools.fit_predict(m_inputs, {'datum': 'ScaledAvg', 'label_encode': 'LesionEncode'}, model, 'PredictionAvg', folds=pred_folds, distribution=model_params)
Tools.fit_predict(m_inputs, {'datum': 'ScaledMax', 'label_encode': 'LesionEncode'}, model, 'PredictionMax', folds=pred_folds, distribution=model_params)
Tools.fit_predict(m_inputs, {'datum': 'ScaledNorm4', 'label_encode': 'LesionEncode'}, model, 'PredictionNorm4', folds=pred_folds, distribution=model_params)
Tools.fit_predict(m_inputs, {'datum': 'ScaledNorm6', 'label_encode': 'LesionEncode'}, model, 'PredictionNorm6', folds=pred_folds, distribution=model_params)
Tools.fit_predict(m_inputs, {'datum': 'ScaledNorm8', 'label_encode': 'LesionEncode'}, model, 'PredictionNorm8', folds=pred_folds, distribution=model_params)
m_inputs[f'Prediction_{Tools.PROBABILITY}'] = m_inputs[f'PredictionAvg_{Tools.PROBABILITY}']

In [18]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'PredictionAvg'
display(HTML(ViewsTools.dataframe_renderer([Views.report(m_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy Avg'])))

name = f'PredictionMax'
display(HTML(ViewsTools.dataframe_renderer([Views.report(m_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy Max'])))

name = f'PredictionNorm4'
display(HTML(ViewsTools.dataframe_renderer([Views.report(m_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy Norm4'])))

name = f'PredictionNorm6'
display(HTML(ViewsTools.dataframe_renderer([Views.report(m_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy Norm6'])))

name = f'PredictionNorm8'
display(HTML(ViewsTools.dataframe_renderer([Views.report(m_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy Norm8'])))

,precision,recall,f1-score,support
Benign,0.72±0.12,0.79±0.12,0.76±0.08,86.00±6.18
Malignant,0.86±0.05,0.81±0.09,0.84±0.03,138.00±4.56
accuracy,0.80±0.04,0.80±0.04,0.80±0.04,0.80±0.04
macro avg,0.79±0.04,0.80±0.03,0.80±0.04,224.00±4.30
weighted avg,0.81±0.01,0.80±0.04,0.81±0.03,224.00±4.30


,precision,recall,f1-score,support
Benign,0.71±0.13,0.66±0.19,0.69±0.12,86.00±6.18
Malignant,0.80±0.08,0.83±0.10,0.82±0.03,138.00±4.56
accuracy,0.77±0.04,0.77±0.04,0.77±0.04,0.77±0.04
macro avg,0.76±0.04,0.75±0.06,0.75±0.06,224.00±4.30
weighted avg,0.77±0.02,0.77±0.04,0.77±0.04,224.00±4.30


,precision,recall,f1-score,support
Benign,0.75±0.10,0.69±0.17,0.72±0.09,86.00±6.18
Malignant,0.81±0.08,0.86±0.10,0.83±0.04,138.00±4.56
accuracy,0.79±0.02,0.79±0.02,0.79±0.02,0.79±0.02
macro avg,0.78±0.02,0.77±0.05,0.77±0.04,224.00±4.30
weighted avg,0.79±0.02,0.79±0.02,0.79±0.03,224.00±4.30


,precision,recall,f1-score,support
Benign,0.70±0.14,0.70±0.19,0.70±0.10,86.00±6.18
Malignant,0.81±0.08,0.81±0.12,0.81±0.03,138.00±4.56
accuracy,0.77±0.03,0.77±0.03,0.77±0.03,0.77±0.03
macro avg,0.75±0.04,0.75±0.06,0.75±0.05,224.00±4.30
weighted avg,0.77±0.03,0.77±0.03,0.77±0.03,224.00±4.30


,precision,recall,f1-score,support
Benign,0.69±0.15,0.69±0.17,0.69±0.10,86.00±6.18
Malignant,0.80±0.07,0.80±0.12,0.80±0.03,138.00±4.56
accuracy,0.76±0.03,0.76±0.03,0.76±0.03,0.76±0.03
macro avg,0.75±0.04,0.75±0.05,0.75±0.05,224.00±4.30
weighted avg,0.76±0.02,0.76±0.03,0.76±0.03,224.00±4.30


# Fusion

In [19]:
inputs = pandas.concat([p_inputs, d_inputs, m_inputs], axis=0)
 # Save
IO.save(inputs, 'Low_Cumulative.pickle')

# Sequential

In [20]:
inputs = IO.load('Low_Cumulative.pickle')

In [21]:
all_image = [True] * len(inputs.index)
single_image = inputs['Modality'] == 'Photography'
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'Prediction_{Tools.PROBABILITY}')
inputs = inputs[single_image].reset_index()

In [22]:
high_folds = [([3],[1]),
             ([4],[2]),
             ([1],[3]),
             ([2],[4])]
single = MultimodalClassifier(method='single', metric=f1_score)
individual = MultimodalClassifier(method='individual', metric=f1_score)

In [23]:
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, single, 'Single')
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, individual, 'Individual') 

In [24]:
IO.save(inputs, 'High_Cumulative.pickle')

# Score

In [25]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load('High_Cumulative.pickle')

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Single'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Test - {name}'])))

name = f'Individual'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))

,precision,recall,f1-score,support
Benign,0.37±0.22,0.13±0.22,0.19±0.17,172.00±12.37
Malignant,0.61±0.10,0.86±0.18,0.72±0.07,276.00±9.11
accuracy,0.58±0.07,0.58±0.07,0.58±0.07,0.58±0.07
macro avg,0.49±0.15,0.50±0.02,0.45±0.09,448.00±8.60
weighted avg,0.52±0.17,0.58±0.07,0.51±0.12,448.00±8.60


,precision,recall,f1-score,support
Benign,0.37±0.22,0.13±0.22,0.19±0.17,172.00±12.37
Malignant,0.61±0.10,0.86±0.18,0.72±0.07,276.00±9.11
accuracy,0.58±0.07,0.58±0.07,0.58±0.07,0.58±0.07
macro avg,0.49±0.15,0.50±0.02,0.45±0.09,448.00±8.60
weighted avg,0.52±0.17,0.58±0.07,0.51±0.12,448.00±8.60
